In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
#alterations = pd.read_csv("data/CTRP/data/v10.A1.enrichment_analysis.txt", sep  = '\t')
alterations = pd.read_csv("data/CTRP v2.0 [2015]/v20.meta.per_cell_line.txt", sep  = '\t')
alterations0 = pd.read_csv("data/CTRP v2.1 [2016]/v21.meta.per_cell_line.txt", sep  = '\t')
alterations1 = pd.read_csv("data/CTRP v2.1 [2016]/v21.meta.gex_features.txt", sep  = '\t')
#alterations2 = pd.read_csv("data/CTRP v2.1 [2016]/v21.meta.per_cell_line.txt", sep  = '\t')
alterations3 = pd.read_csv("data/CTRP v2.1 [2016]/v21.meta.per_compound.txt", sep  = '\t')
print(len(alterations.index))
print(len(alterations0.index))
#print(alterations1.columns)
#print(alterations2.columns)
#print(alterations3.columns)
#alterations4 = pd.read_csv("data/mutations.txt", sep  = '\t')
alterations.head()

1107
835


,master_ccl_id,ccl_name,ccl_availability,ccle_primary_site,ccle_primary_hist,ccle_hist_subtype_1
0,1,697,ccle;public,haematopoietic_and_lymphoid_tissue,lymphoid_neoplasm,acute_lymphoblastic_B_cell_leukaemia
1,3,5637,ccle;public,urinary_tract,carcinoma,NaN
2,4,2313287,ccle;public,stomach,carcinoma,adenocarcinoma
3,5,1321N1,ccle,central_nervous_system,glioma,astrocytoma
4,6,143B,ccle,bone,osteosarcoma,NaN


In [ ]:
#response = pd.read_csv("data/CTRP/data/v10.D3.area_under_conc_curve.txt", sep  = '\t')
#response = pd.read_csv("data/CTRP v2.0 [2015]/v20.data.curves_post_qc.txt", sep  = '\t')
response = pd.read_csv("data/CTRP v2.1 [2016]/v21.data.auc_sensitivities.txt", sep  = '\t')
response['Response'] = pd.cut(response['area_under_curve'], bins=[0, 2.5, 5.5, 50], labels=['Sensitive','Medium','Non-responsive'])
response.head()

In [ ]:
print(f"Number of cancer cell lines: {len(alterations['ccl_name'].unique())}")
print(f"Number of compounds: {len(alterations['cpd_name'].unique())}")

In [ ]:
response = pd.read_csv('data/Drug_response.txt', sep = '\t')
response.head()

In [ ]:
#Let's look at the alterations dataset

print("Are there any NaN values in the mutations dataset? ", alterations.isnull().values.any()) #no Nan values
print(f"Size of the dataset: {alterations.shape}: {alterations.shape[0]} mutations available in {alterations.shape[1]-1} cell populations.")

In [ ]:
#Let's look at the response dataset

print("Are there any NaN values in the response dataset? ", response.isnull().values.any()) #no Nan values
print(f"Size of the dataset: {response.shape}: the response to {response.shape[0]} drugs in {response.shape[1]} cell populations.")

In [ ]:
proportion_non_nans = (response.count()/response.count()["Description"]).sort_values() #looks at the proportions of NaNs for each cell population
proportion_non_nans

In [ ]:
weakest_response = min(response.drop('Description', axis = 1).min())
print(weakest_response) #No response is 0 ! 

In [ ]:
#What is the proportion of NaNs? Fill
 
filled_response = response.fillna(-1)
liste = range(0,len(filled_response.columns))
filled_response = filled_response.reset_index(drop = True).drop("Description", axis = 1).T.reset_index(drop=True).T#.rename(columns = map(lambda elem: elem, liste))# for i in len(filled_response))
fig = plt.figure( figsize = (15,10))
sns.heatmap(filled_response, cmap = sns.cm.rocket)
plt.xlabel('Cell populations')
plt.ylabel('Drugs');

### Transpose dataframes for homogeneity

In [ ]:
alterations_t = alterations.set_index('Description').transpose()
response_t = response.set_index('Description').transpose()

print('Alterations:')
display(alterations_t.head())

print('Responses:')
response_t.head()

#### Let's first try to see if the number of mutations has any impact on the response (naive)

In [ ]:
alterations_t['Number of mutations'] = list(alterations_t.iloc[:,:].sum(axis = 1))

In [ ]:
print(f'Number of cells in the alterations database: {len(alterations_t.index)}')
print(f'Number of cells in the response database: {len(response_t.index)}')
alterations_t.head()

##### Let's look at the elements in common (these are the only ones that can be used for training)

In [ ]:
df1 = pd.concat([response_t, alterations_t['Number of mutations']], axis = 1, join = 'inner')
df1.head()

In [ ]:
print(f'Number of cell populations in common: {len(df1.index)}')
proportion_non_nan_drugs = (df1.count()/len(df1.index)).sort_values() #looks at the proportions of NaNs for each cell population
#noNanDrugs = [drug for drug in df1.columns if df1[drug].isnull().values.any() == False]
proportion_non_nan_drugs #we have NaN's in all columns ! What shall we do with them?

In [ ]:
sns.scatterplot(df1['Number of mutations'], df1['(5Z)-7-Oxozeaenol']);

In [ ]:
#Let's try to plot the density function of the first drug as a function of the number of mutations

sns.jointplot(x ='Number of mutations', y = str(df1.columns[0]), data = df1.dropna(), kind = 'kde'); #for now, just dropping all the NaNs.

In [ ]:
sns.scatterplot(df1['Number of mutations'], df1['Bleomycin (50 uM)']);

In [ ]:
#Let's try to plot the density function of the first drug as a function of the number of mutations

sns.jointplot(x ='Number of mutations', y = 'Bleomycin (50 uM)', data = df1.dropna(), kind = 'kde'); #for now, just dropping all the NaNs.